In [1]:
pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt import Optimizer
from skopt.learning import GaussianProcessRegressor
from skopt.acquisition import gaussian_ei

In [3]:
# Prepare the input data
# X = np.array([[2, 5, 13], [20, 15, 6.5], [2, 5, 6.5], [2, 15, 13], [10, 5, 13], [10, 15, 6.5], [10, 5, 6.5], [10, 15, 13], [20, 5, 13], [2, 15, 6.5], [20, 5, 6.5], [20, 15, 13], [0, 0, 0]])
Y = np.array([20.9, 27.4, 29.7, 34.2, 30.1, 29.6, 41.7, 21.8, 28.1, 42, 32.2, 53.8, 18.7])
X = np.array([[2, 5], [20, 15], [2, 5], [2, 15], [10, 5], [10, 15], [10, 5], [10, 15], [20, 5], [2, 15], [20, 5], [20, 15], [0, 0]])


In [4]:
# def objective(x):
#     x = np.reshape(x, (1, -1))
#     dist = np.linalg.norm(X - x, axis=1)
#     weights = np.exp(-dist)  # Exponential weights based on distance
#     weighted_sum = np.sum(Y * weights)  # Weighted sum of Y values
#     return weighted_sum  # Return the negative of the corresponding Y value


In [5]:
# Define the bounds for each dimension
bounds = [(0, 100), (0, 100)]

In [6]:
# Define the Gaussian process surrogate model
surrogate_model = GaussianProcessRegressor()

In [7]:
# Fit the surrogate model to the available data
model = surrogate_model.fit(X, Y)

In [8]:
# Initialize the Bayesian optimizer
optimizer = Optimizer(bounds, base_estimator=surrogate_model, acq_func="EI", acq_optimizer="sampling")

In [9]:
# Track the best objective value and the corresponding parameters
best_objective = -np.inf
best_params = None

# Store the best objective value at each iteration
best_objectives = []

In [10]:
for i in range(100):
    # Select the next point to evaluate
    x_next = optimizer.ask()

    # Convert x_next to a numpy array and reshape it
    x_next = np.array(x_next).reshape(1, -1)

    # Evaluate the objective function at the selected point
    y_next = surrogate_model.predict(x_next.reshape(1, -1))

    # y_next = np.array(y_next).reshape(1, -1)

    # # Update the best objective value and parameters if a new best is found
    # if y_next >= best_objective:
    #     best_objective = y_next
    #     best_params = x_next

   # Provide the observation to the optimizer
    optimizer.tell(list(x_next.flatten()), list(y_next.flatten()))


    # # Print the current iteration and the best objective value found so far
    # print("Iteration:", i + 1)
    # print("Best Objective Value:", best_objective)

    # Print the current iteration and the maximum observed value found so far
    print("Iteration:", i + 1)
    print("Maximum Observed Value:", optimizer.yi.max())

    # # Plot the current best solution
    # x_vals = np.array(optimizer.Xi)[:, 0]
    # y_vals = np.array(optimizer.Xi)[:, 1]
    # z_vals = np.array(optimizer.Xi)[:, 2]
    # objective_vals = -np.array(optimizer.yi)

    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # ax.scatter(x_vals, y_vals, z_vals, c=objective_vals, cmap='viridis')
    # ax.set_xlabel('X')
    # ax.set_ylabel('Y')
    # ax.set_zlabel('Z')
    # ax.set_title('Iteration ' + str(i + 1))
    # plt.show()

ValueError: ignored

In [ ]:
# Plot the final solution
x_vals = np.array(optimizer.Xi)[:, 0]
y_vals = np.array(optimizer.Xi)[:, 1]
z_vals = np.array(optimizer.Xi)[:, 2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_vals, y_vals, z_vals, c=optimizer.yi, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Final Solution')

plt.show()

In [ ]:
print("Best Parameters:", best_params)
print("Maximum Value of Y:", best_objective)